
# Debugging LLM Chains: A Practical Guide

When working with large language models (LLMs) like GPT-4 and beyond, engineers often face the challenge of understanding the data flow and decision paths taken by the model. One critical tool in this endeavor is the tracing of LLM chains, akin to debugging in traditional software development. By tracing these chains, we can isolate specific points in the model's operations where things might go awry, or simply gain deeper insights into its behavior. In this article, we'll break down the nuts and bolts of this process, offering practical steps and tools for efficient and effective tracing. Let's dive in.


## Introducing Our Coding Exercise LLM Chain

We're engineering an LLM chain designed to serve up coding exercises tailored to student needs. Here's the workflow: 
1. First, we interpret the student's query. 
2. Next, we pull existing user data. 
3. We use LLM to combine existing user data with the new query to keep track of user's progress
4. This data is fed into our LLM to formulate a customized coding assignment. 

With multiple steps in the mix, things can occasionally go sideways. Tracing becomes paramount, not just for pinpointing where a potential breakdown might occur, but also for ensuring that each step seamlessly leads to the next. In this article, we'll walk through how to implement a robust tracing mechanism to keep this workflow smooth and efficient.

## OpenAI Login
We'll use OpenAI API in this tutorial, so we need to set the `OPENAI_API_KEY' environment variable first. 

In [1]:
from getpass import getpass
import openai
import os

if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


## Store user data
We need to persist the status of our users, so that we can serve them customized assignments. Here, we'll go for a simplistic solution and simply store it in a text file. 

In [2]:
def retrieve_user_data(user_id: str, directory_path: str = 'user_data/') -> dict:
    """
    Fetch user data from a text file based on user_id.
    If no data is found for a user, return a default string.
    """
    file_path = os.path.join(directory_path, f"{user_id}.txt")
    
    # If the file exists for the user, read and return the content.
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            user_status_summary = file.read().strip()
        return user_status_summary
    # If no file/data is found for the user, return a default value.
    else:
        return 'No previous data found.'

# Usage example:
print(retrieve_user_data("user123"))

The user found the previous assignment too challenging and is requesting help. They are given a step-by-step breakdown of the problem and code implementation. The user is advised to test the function with different inputs and is encouraged to ask for further assistance if needed.


## LLM API
We'll use a simple wrapper on top of the `openai.ChatCompletion` API, and decorate it to handle rate limits gracefully. 

In [3]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  

MODEL_NAME = "gpt-3.5-turbo"

system_prompt = "You are an AI tutor helping students prepare for machine learning coding interviews."

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

def llm(system_prompt, user_prompt, n=1):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    return responses.choices[0].message.content

user_prompt = "I want a challenging Python exercise."
print(llm(system_prompt, user_prompt))

Sure! Here's a challenging Python exercise for you:

Write a function called `common_elements` that takes two lists as input and returns a new list containing the elements that are common between the two input lists. The order of the elements in the output list doesn't matter.

For example, if the input lists are `[1, 2, 3, 4]` and `[3, 4, 5, 6]`, the output should be `[3, 4]`.

You are not allowed to use any built-in Python functions or libraries that directly solve this problem. You must implement the solution yourself.

Try to come up with an efficient solution with a time complexity better than O(n^2), if possible.


## Generating Assignments
We can now put together the user summary we retrieved from our storage, combine it with user query, and use LLM to generate an assignment. 

In [4]:
def generate_assignment(user_summary: str, user_query: str) -> str:
    """
    Based on user's summary and query, use LLM to generate a coding assignment.
    """
    combined_prompt = f"This is the information we know about the user: {user_summary}\nThis is the user query: {user_query}"
    return llm(system_prompt, combined_prompt)

# Usage example:
user_data = retrieve_user_data("user123")
user_query = "I want a challenging Python exercise."
print(generate_assignment(user_data, user_query))

Sure! How about trying to solve the following problem:

Write a Python function called "unique_characters" that takes in a string as input and returns True if all the characters in the string are unique (i.e., no character appears more than once), and False otherwise. The function should be case-sensitive.

For example:
- unique_characters("abcde") should return True
- unique_characters("aabbcc") should return False
- unique_characters("Hello") should return True
- unique_characters("Python") should return False

To solve this problem, you can follow these steps:
1. Initialize an empty set called "char_set" to store the unique characters.
2. Iterate over each character "ch" in the input string.
3. Inside the loop, check if the character "ch" is already in the "char_set". If it is, return False because it means the character is repeated.
4. If the character "ch" is not in the "char_set", add it to the set.
5. After the loop, if all the characters have been processed without any repeats,

## Update user state
We have more information about the user now - the query we got from them, and the assignment we proposed. We need to update the our user data with this new information. 

In [6]:
def summarize_data(user_id: str, previous_summary: str, user_query: str, generated_assignment: str, directory_path: str = 'user_data/') -> str:
    """
    Request a new summary using LLM based on the previous summary, user query, and generated assignment.
    Save the new summary to the user's file, overwriting its previous contents.
    """
    system_prompt = "You are responsible for accurately and concisely summarizing user information that can be used later on to generate appropriate coding assignments for students, track progress etc."
    # Combine inputs to form the prompt for LLM
    combined_prompt = f"Previous Summary: {previous_summary}\nUser Query: {user_query}\nGenerated Assignment: {generated_assignment}\nPlease summarize this information."
    
    # Request a new summary using the llm function
    new_summary = llm(system_prompt, combined_prompt)

    # Save the new summary to the user's file
    file_path = os.path.join(directory_path, f"{user_id}.txt")
    with open(file_path, 'w') as file:
        file.write(new_summary)

    return new_summary

# Usage example:
user_data = retrieve_user_data("user123")
user_query = "I want a challenging Python exercise."
generated_assignment = generate_assignment(user_data, user_query)
new_summary = summarize_data("user123", user_data, user_query, generated_assignment)
print(new_summary)

The user requested a challenging Python exercise. An assignment was generated that involves finding all unique triplets in a given list of integers that sum up to zero. The function `find_triplets` needs to be implemented to solve this problem. The generated exercise includes an example, a note, and a step-by-step breakdown of how to implement the solution. The user is encouraged to test the function with different inputs and ask for further assistance if needed.


## Putting things together
We have all the building blocks now to get an assignment based on a user query. Let's see what we get!

In [8]:
def get_assignment(user_id: str, user_query: str, directory_path: str = 'user_data/') -> str:
    """
    Central function to interpret the user query, fetch user data, generate assignment, and summarize the data.
    """
    # Retrieve the user's previous data
    previous_summary = retrieve_user_data(user_id, directory_path)

    # Generate a coding assignment based on the user's previous summary and query
    generated_assignment = generate_assignment(previous_summary, user_query)
    
    # Summarize the data and save the new summary for the user
    summarize_data(user_id, previous_summary, user_query, generated_assignment, directory_path)
    
    return generated_assignment

# Usage example:
resulting_assignment = get_assignment("user123", "That was too hard for me.")
print(resulting_assignment)


No worries! I'm here to help. Let's start by breaking down the problem into smaller steps. Remember, practice makes perfect, so even if it feels challenging, it's a great opportunity to learn.

To solve this problem, we need to find all unique triplets in a given list of integers that sum up to zero. Here's the step-by-step breakdown:

1. Sort the input list in ascending order.
   - This will help us in identifying unique triplets efficiently.

2. Initialize an empty result list to store the triplets.

3. Iterate through the input list, considering each element as the first element of the triplet.

4. For each first element, use a two-pointer approach to find the remaining two elements that sum up to the negative of the first element.
   - Let's say the current element is `num`. We need to find two elements in the remaining part of the list that sum up to `-num`.

5. Determine the left and right pointers. The left pointer should start from the next element after `num`, and the right po

## Why Tracing Matters

By now, you've probably noticed how we've integrated W&B Traces into the code. This straightforward addition holds immense value for software engineers. It effectively saves all essential data—computation inputs, outputs, timing, and metadata—in a structured `StreamTable`. This practical functionality enables you to easily monitor, debug, and analytically dissect your code's behavior during both development and production. No complex steps required—just click the link to weave.wandb.ai. There, you can conveniently create a default Trace Debug Board using the right-hand panel and explore your Traces visually or craft your own custom charts. It's a practical, hands-on approach to enhancing your code analysis and debugging process.